# 🛠️ Паттерн проектирования AI-агентов "Tool Use"

---

**Tool Use Pattern** — это архитектурный паттерн, при котором LLM расширяет собственные способности за счёт использования внешних инструментов (tools). Под "инструментами" понимаются функции, API, системы поиска, кода, базы данных и т.д., вызываемые агентом через текстовую или программную инструкцию.

---

## Зачем?

LLM по своей природе ограничены:

- не имеют доступа к реальному миру,
- не могут выполнять действия,
- не обладают долговременной памятью,
- склонны к галлюцинациям.

Tool use позволяет "превратить LLM в агента", снабдив его следующими возможностями:

| Способность          | Пример инструмента         |
|----------------------|----------------------------|
| Доступ к интернету   | Web Search                 |
| Выполнение кода      | Python Executor            |
| Вызов API            | Weather API, Email API     |
| Работа с памятью     | VectorDB (Retrieval)       |
| Навигация по данным  | SQL, PDF parser, RAG       |

---


# Паттерн проектирования AI-агентов "Рефлексия"

---

## 📌 Введение

Рефлексия (reflection) — это паттерн в проектировании  ai агентов, при котором агент анализирует собственное поведение, знание, ошибки и стратегии, чтобы модифицировать или оптимизировать свои действия. В отличие от react агентов, рефлексивный агент имеет модель самого себя и способность к самоанализу.

Этот паттерн перекликается с идеями метапознания, саморегуляции и интроспекции в когнитивной науке.

---

### основа лежит в когнитивной психологии

## **Важно**: Reflection превращает статичную LLM в динамичную систему, способную учиться "на лету". Для успешной реализации нужны:

- Четкие критерии оценки.

- Механизмы предотвращения "бесконечных петлей".

- Контекстная память для долгосрочного улучшения.

### Рефлексия на LangChain

### 📦 **Шаг 1: Задача — Агент решает задачу**

In [54]:
# from langchain_experimental.tools.python.tool import PythonREPLTool
# from langchain.agents import initialize_agent

# py_agent = initialize_agent(
#     tools=[PythonREPLTool()],
#     llm=llm,
#     verbose=True,
#     handle_parsing_errors=True,
#     max_iterations=3
# )

In [55]:
# list_ = [7, 9, 5, 3, 6]
# solution = py_agent.run(f'Отсортируй список с помощью bubble sort: {list_}')

In [19]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.llms import Ollama

llm = Ollama(model="mistral-nemo:12b-instruct-2407-q3_K_L", temperature=0.4)

task_prompt = PromptTemplate.from_template("Ты python-разработчик. Реши задачу: {task}")
chain = LLMChain(llm=llm, prompt=task_prompt)

task = "Напиши функцию на Python для сортировки списка чисел по возрастанию"
solution = chain.run(task)

print(solution)

Вот функция, которая sorts a list of numbers in ascending order using the built-in `sorted()` function:

```python
def sort_numbers_ascending(numbers):
    return sorted(numbers)
```

Вы можете использовать эту функцию следующим образом:

```python
numbers = [3, 1, 4, 1, 5, 9, 2]
sorted_numbers = sort_numbers_ascending(numbers)
print(sorted_numbers)  # Output: [1, 1, 2, 3, 4, 5, 9]
```

Если вы хотите написать свою собственную функцию сортировки без использования встроенных функций Python (таких как `sorted()` или `sort()`), вот пример функции сортировки пузырьком:

```python
def bubble_sort(numbers):
    n = len(numbers)
    for i in range(n-1):
        for j in range(0, n-i-1):
            if numbers[j] > numbers[j+1]:
                numbers[j], numbers[j+1] = numbers[j+1], numbers[j]
    return numbers
```

Вы можете использовать эту функцию следующим образом:

```python
numbers = [3, 1, 4, 1, 5, 9, 2]
sorted_numbers = bubble_sort(numbers)
print(sorted_numbers)  # Output: [1, 1, 2,

### 🪞 **Шаг 2: Рефлексия — тот же агент оценивает свое решение**

In [52]:
reflection_prompt = PromptTemplate.from_template(
    "Проанализируй следующий код. Есть ли ошибки? Что можно улучшить?\n\n{code}"
)
reflection_chain = LLMChain(llm=llm, prompt=reflection_prompt)

reflection = reflection_chain.run(solution)
print("Рефлексия:\n", reflection)

Рефлексия:
 Указанный код является списком в языке программирования Python и не содержит ошибок синтаксиса или семантических ошибок. Это просто список из чисел: 3, 5, 6, 7 и 9.

Однако, если вы хотите проанализировать этот список на предмет каких-либо других проблем или возможностей для улучшения, то это зависит от того, как именно вы планируете использовать этот список. Например:

1. Если вам нужно сохранить эти числа в переменных, то возможно было бы лучше создать кортеж (tuple) вместо списка, так как кортежи являются неизменяемыми и занимают меньше памяти.
2. Если вам нужно отсортировать или отфильтровать эти числа, то было бы полезно использовать функции Python для сортировки или фильтрации списков.
3. Если вам нужно вычислить среднее значение этих чисел, то можно использовать функцию `sum()` и `len()` для подсчета суммы всех чисел в списке и количества чисел соответственно, а затем разделить сумму на количество чисел.

В целом, этот код является простым и не содержит явных ошибок 

# 🔁 Паттерн проектирования AI-агентов "ReAct"

---

## 📌 ReAct

ReAct (Reason + Act) — это паттерн проектирования AI-агентов, при котором языковая модель чередует шаги рассуждения и действия, чтобы решать задачи в реальном или симулированном мире. В этом подходе LLM действует как агент, который:

1. обдумывает, что нужно сделать (Reasoning),
2. действует через вызов внешнего инструмента или среды (Action), 
3. наблюдает за результатом (Observation),
4. и повторяет процесс, пока не достигнет цели.

## Впервые было описано в статье: 

**ReAct: Synergizing Reasoning and Acting in Language Models**  
🧑‍🔬 Yao et al., Google Research, 2022  
[arXiv:2210.03629](https://arxiv.org/abs/2210.03629)
---

## зачем?

Традиционные LLM могут:

- рассуждать логически (Chain-of-Thought),
- использовать инструменты (Tool Use),

но не умеют чередовать их. ReAct позволяет:

- комбинировать reasoning и вызовы API,
- пошагово уточнять стратегию,
- взаимодействовать с окружающей средой,
- улучшать интерпретируемость.

---

In [31]:
import requests
import math
from langchain.agents import Tool, initialize_agent, AgentType
from langchain_community.llms import Ollama
from langchain.utilities import WikipediaAPIWrapper

## 🔍 Инструмент 1: DuckDuckGo поиск

In [32]:
def duckduckgo_search(query: str) -> str:
    url = "https://api.duckduckgo.com/"
    params = {
        "q": query,
        "format": "json",
        "no_redirect": 1,
        "no_html": 1
    }
    try:
        res = requests.get(url, params=params)
        data = res.json()
        if data.get("AbstractText"):
            return data["AbstractText"]
        elif data.get("RelatedTopics"):
            topics = data["RelatedTopics"]
            if isinstance(topics, list) and len(topics) > 0:
                topic = topics[0]
                if isinstance(topic, dict) and "Text" in topic:
                    return topic["Text"]
        return "Информация не найдена"
    except Exception as e:
        return f"Ошибка при поиске: {e}"

duckduckgo_tool = Tool(
    name="DuckDuckGo",
    func=duckduckgo_search,
    description="Поиск фактической информации через DuckDuckGo"
)

## 🧮 Инструмент 2: Калькулятор

In [40]:
from langchain.chains import LLMMathChain

llm_math = LLMMathChain.from_llm(llm=llm)

calc_tool = Tool(
    name='Калькулятор',
    func=llm_math.run,
    description='Может производить математические расчёты.'
)

In [41]:

# def calculator_tool(expression: str) -> str:
#     try:
#         result = eval(expression, {"__builtins__": {}}, math.__dict__)
#         return str(result)
#     except Exception as e:
#         return f"Ошибка в вычислении: {e}"

# calc_tool = Tool(
#     name="Calculator",
#     func=calculator_tool,
#     description="Выполняет математические выражения, например: '2**8 + sqrt(16)'"
# )

## 📚 Инструмент 3: Википедия

In [42]:
wiki = WikipediaAPIWrapper()

wiki_tool = Tool(
    name="Wikipedia",
    func=wiki.run,
    description="Поиск энциклопедической информации через Википедию")

## ⚙️ Инициализация ReAct-агента с тремя инструментами

In [43]:
llm = Ollama(model="mistral-nemo:12b-instruct-2407-q3_K_L", temperature=0.4)

agent = initialize_agent(
    tools=[duckduckgo_tool, calc_tool, wiki_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [44]:
response = agent.run("Сколько будет (4.5*2.1)+2.2?")
print(response)



> Entering new AgentExecutor chain...
I need to perform a mathematical calculation.
Action: Калькулятор
Action Input: 4.5 * 2.1 + 2.2
Observation: Answer: 11.650000000000002
Thought:I now know the final answer
Final Answer: 11.65

> Finished chain.
11.65


In [45]:
response = agent.run("Кто такой Альберт Эйнштейн?")
print(response)



> Entering new AgentExecutor chain...
Я должен начать с поиска информации об Альберте Эйнштейне в Википедии.
Action: Wikipedia
Action Input: Albert Einstein
Observation: Page: Albert Einstein
Summary: Albert Einstein (14 March 1879 – 18 April 1955) was a German-born theoretical physicist who is best known for developing the theory of relativity. Einstein also made important contributions to quantum mechanics. His mass–energy equivalence formula E = mc2, which arises from special relativity, has been called "the world's most famous equation". He received the 1921 Nobel Prize in Physics for his services to theoretical physics, and especially for his discovery of the law of the photoelectric effect.
Born in the German Empire, Einstein moved to Switzerland in 1895, forsaking his German citizenship (as a subject of the Kingdom of Württemberg) the following year. In 1897, at the age of seventeen, he enrolled in the mathematics and physics teaching diploma program at the Swiss federal polyt

# Паттерн проектирования AI-агентов "Planning"

---

## 📌 Planning

Planning — это паттерн проектирования, при котором агент разбивает задачу на подзадачи, формирует последовательность шагов, а затем поэтапно их выполняет. Этот подход позволяет LLM-агентам решать многошаговые, долгосрочные или сложноструктурированные задачи, выходящие за пределы одного запроса-ответа.

---

## Зачем?

| Без плана                          | С планированием                           |
|------------------------------------|-------------------------------------------|
| Агент действует жадно и локально   | Агент оптимизирует глобальную стратегию   |
| Пропускает важные шаги             | Явно их включает и проверяет выполнение   |
| Не видит цели как целое            | Разбивает цель на управляемые единицы     |
| Нет повторного использования шагов | Возможен reuse/кэширование подзадач       |

---

## 🧠 Когнитивные аналоги

| В человеке                   | В LLM-агенте                    |
|------------------------------|----------------------------------|
| Установка целей              | Goal inference / input prompt   |
| Разбиение на подцели         | Chain-of-Thought / Task Decomp  |
| Составление плана действий   | LLM-планировщик (Planner)       |
| Мониторинг выполнения        | Execution agent + Memory        |

---


In [ ]:
import os
import requests
from amadeus import Client, ResponseError
from langchain.agents import Tool, initialize_agent, AgentType
from langchain_community.llms import Ollama
from langchain.utilities import WikipediaAPIWrapper

In [37]:
import os
os.environ["AMADEUS_CLIENT_ID"] = "hS74sQ71STZ8ASGpSfCOOdUX7DNV1Q5c"
os.environ["AMADEUS_CLIENT_SECRET"] = "4VAv0oQ5bhQ5DjID"

In [ ]:
from amadeus import Client

amadeus = Client(
    client_id=os.getenv("AMADEUS_CLIENT_ID"),
    client_secret=os.getenv("AMADEUS_CLIENT_SECRET")
)

[{'type': 'location', 'subType': 'CITY', 'name': 'PARIS', 'detailedName': 'PARIS/FR', 'id': 'CPAR', 'self': {'href': 'https://test.api.amadeus.com/v1/reference-data/locations/CPAR', 'methods': ['GET']}, 'timeZoneOffset': '+02:00', 'iataCode': 'PAR', 'geoCode': {'latitude': 48.85334, 'longitude': 2.34889}, 'address': {'cityName': 'PARIS', 'cityCode': 'PAR', 'countryName': 'FRANCE', 'countryCode': 'FR', 'regionCode': 'EUROP'}, 'analytics': {'travelers': {'score': 68}}}, {'type': 'location', 'subType': 'CITY', 'name': 'LE TOUQUET PARIS PLAGE', 'detailedName': 'LE TOUQUET PARIS PLAGE/FR', 'id': 'CLTQ', 'self': {'href': 'https://test.api.amadeus.com/v1/reference-data/locations/CLTQ', 'methods': ['GET']}, 'timeZoneOffset': '+02:00', 'iataCode': 'LTQ', 'geoCode': {'latitude': 50.5175, 'longitude': 1.62056}, 'address': {'cityName': 'LE TOUQUET PARIS PLAGE', 'cityCode': 'LTQ', 'countryName': 'FRANCE', 'countryCode': 'FR', 'regionCode': 'EUROP'}, 'analytics': {'travelers': {'score': 2}}}, {'type

In [ ]:
def get_weather(_: str) -> str:
    lat, lon = 48.8566, 2.3522
    try:
        res = requests.get(
            "https://api.open-meteo.com/v1/forecast",
            params={"latitude": lat, "longitude": lon, "current_weather": True}
        ).json()
        cw = res.get("current_weather", {})
        return f"Сейчас в Париже: {cw.get('temperature')}°C, ветер {cw.get('windspeed')} км/ч, время: {cw.get('time')}"
    except Exception as e:
        return f"Не удалось получить погоду: {e}"

wiki = WikipediaAPIWrapper()
def get_attractions(_: str) -> str:
    return wiki.run("Tourist attractions in Paris")

def get_hotels(_: str) -> str:
    try:
        response = amadeus.reference_data.locations.hotels.by_geocode.get(
            latitude=48.8566, longitude=2.3522
        )
        hotels = response.data[:5]
        return "\n".join([
            f"🏨 {h['name']}, расстояние: {h.get('distance', 'N/A')} км" for h in hotels
        ])
    except ResponseError as e:
        return f"Ошибка запроса: {e}"

In [ ]:
llm = Ollama(model="mistral-nemo:12b-instruct-2407-q3_K_L", temperature=0.4)

tools = [
    Tool("Weather", get_weather, "Погода в Париже", return_direct=True),
    Tool("Attractions", get_attractions, "Что посмотреть в Париже"),
    Tool("Hotels", get_hotels, "Отели в центре Парижа", return_direct=True)
]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

prompt = (
    "Спланируй поездку в Париж на выходные. \n"
    "Сначала покажи погоду, затем 3–4 достопримечательности, затем отели. Не повторяй шаги."
)
agent.run(prompt)